# Plots for Icarus paper <a class="anchor" id="top"></a>


### Example run
These plots are for runs which are comparable to Case 1 of instantaneous accretion in [Dodds et. al. 2021](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2020JE006704). r=500km, $t_{acc}$=0.8 Ma after CAI formation, $X_{S,0}$=[27.1,30.05,33] wt%, all other parameters are median values from Sanderson et. al. 2024 (EPSL. in review). For runs 1-6, odd number runs have $m_{frac}=0$ and even number runs have $m_{frac}=1$. Run 7 is for a 400km planetesimal accreted at 0.5Ma after CAI formation for comparison with the single accretion event model in [Bryson et. al. 2019](https://doi.org/10.1016/j.epsl.2019.05.046)

+ [Summary stats](#stats)
+ [Temperature profile](#temp)
+ [Heat fluxes](#flux)
+ [Field strength and Reynolds number](#Bfield)
+ [Solidification endmembers](#icfrac)

### Core solidification discussion
+ [Convective lengthscales](#conv)
    + [Ratio of lengthscales to B and Rem](#ratio)
+ [Thermal vs compositional dynamo field strength](#therm-comp)


### Set-up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.append('../')
from load_info import load_run_info, load_run_results

#scale time to Myr
from plot_params import Myr, f0

Choose run for main thermal plots and where to save files.

Run 4 has $m_{frac}$=1 and the median value of $X_{S,0}$ so use as default for plotting.

In [ ]:
run=4
save = True# do you want to save your figures?
automated = True
log_time = True #do you want to plot time logarithmically
path = '../Results_combined/Dodds_comp/' #path to files 
save_path = '../Plots/Icarus_paper/'

Load results for differentiation

In [ ]:
npzfile = np.load(f'{path}run_{run}_diff.npz')
Tdiff = npzfile['Tdiff'] 
iron = npzfile['Xfe']
silicate_d = npzfile['Xsi']
tdiff = npzfile['t_diff']
Ra_d = npzfile['Ra']
Ra_crit_d = npzfile['Ra_crit']
d0_diff = npzfile['d0']

Load results for thermal evolution

In [ ]:
npzfile = np.load(f'{path}run_{run}.npz')
Tc= npzfile['Tc'] 
Tc_conv = npzfile['Tc_conv']
Tcmb = npzfile['Tcmb']
Tm_mid = npzfile['Tm_mid']
Tm_conv = npzfile['Tm_conv']
Tm_surf = npzfile['Tm_surf'] 
T_profile = npzfile['T_profile']
f = npzfile['f'] 

t = npzfile['t'] #time in s
Rem = npzfile['Rem'] # magnetic Reynolds number 
B = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Flux = npzfile['Flux']
Ra = npzfile['Ra'] 
RaH = npzfile['RaH'] 
RanoH = npzfile['RanoH'] 
Racrit = npzfile['Racrit'] 
d0 = npzfile['d0'] 
min_unstable = npzfile['min_unstable'] 
Urey = npzfile['Ur']
Xs = npzfile['Xs']
dl = npzfile['dl']
dc = npzfile['dc']
Fs = Flux[0]
Fcmb = Flux[1]
Fad = Flux[2]
Frad = Flux[3]

Concatenate shared variables

In [ ]:
Tall = np.hstack((Tdiff,np.transpose(T_profile)))
tall = np.append(tdiff,t)
iron_all = np.hstack
Ra_all = np.append(Ra_d,Ra)
Ra_crit_all = np.append(Ra_crit_d,Racrit)
d0_all = np.append(d0_diff,d0)

Scale time by Myr

In [ ]:
t_plot_all = tall/Myr
t_plot_t = t/Myr

Run info

In [ ]:
if automated == True:
    r, dr, tstart, dt, viscosity, icfrac = load_run_info(run,f'{path}auto_params.csv')
else: 
    r, dr, tstart, dt, viscosity, icfrac = load_run_info(run,f'{path}run_info.csv')

## Summary statistics <a class="anchor" id="stats"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Load in results data

In [ ]:
results = load_run_results(run,f'{path}run_results.csv')

Load in run parameters

In [ ]:
if automated == True:
    params = pd.read_csv(f'{path}auto_params.csv',skiprows=[1])
    params = params[params['run']==run]
else:
    params = pd.read_csv(f'{path}run_info.csv',skiprows=[1])
    params = params[params['run']==run]

Assign to variables values that will be used later

In [ ]:
nmantle = int((r/dr)/2)
terode = results.at[0,'terode']
tstrat_remove = results.at[0,'tstrat_remove']
diff_time = results.at[0,'diff_time']
peak_coreT = np.amax(Tall[:nmantle,:])
loc_max = np.where(Tall[:nmantle,:]==peak_coreT)[1][0] #take the set of time coordinates and first value (they should all be the same)
tcoremax = tall[loc_max]/Myr
fcond_t = results.at[0,'fcond_t']
diff_T =results.at[0,'diff_T']

#dynamo on and off times
var_results = pd.read_csv(path+'run_results.csv',skiprows=[1])
on1=var_results.loc[var_results['run']==run,'magon_1'].values[0]
off1=var_results.loc[var_results['run']==run,'magoff_1'].values[0]
on2=var_results.loc[var_results['run']==run,'magon_2'].values[0]
off2=var_results.loc[var_results['run']==run,'magoff_2'].values[0]
on3=var_results.loc[var_results['run']==run,'magon_3'].values[0]
off3=var_results.loc[var_results['run']==run,'magoff_3'].values[0]

Print a summary

In [ ]:
print(f"Differentiation is at {results.at[0,'diff_time']:.2f} Myr")
print(f"The temperature at differentiation is at {results.at[0,'diff_T']:.2f}K")
print(f"Peak magma ocean temp is {results.at[0,'peakT']:.0f}K at {results.at[0,'tmax']:.2f} Myr")
print(f"Stratification starts at {results.at[0,'tstrat_start']:.2f} Myr")
print(f"Mantle hotter than the core until {results.at[0,'tstrat_remove']:.2f} Myr")
print(f"Erosion of stratification by {results.at[0,'terode']:.2f} Myr") 
print(f"End of mantle convection by {results.at[0,'fcond_t']:.2f} Myr")
print(f"The maximum thermal Rem is {results.at[0,'max_Rtherm']:.2f} at {results.at[0,'max_Rtherm']:.2f} Myr")
print(f"The maximum thermal field strength is {results.at[0,'max_Btherm']:.2e}T at {results.at[0,'max_Bthermt']:.2f} Myr")
print(f"The maximum compositional Rem is {results.at[0,'max_Rcomp']:.2f}")
print(f"The maximum compositional field strength is {results.at[0,'max_Bcomp']:.2e}")
print(f'The dynamo starts at {on1:.2f} Myr, stops at {off1:.2f} Myr and lasts {off1-on1:.2f} Myr')
if on2 > 0:
    print(f'The second dynamo starts at {on2:.2f} Myr, stops at {off2:.2f} Myr and lasts {off2-on2:.2f} Myr')
if on3 > 0:
    print(f'The third dynamo starts at {on3:.2f} Myr, stops at {off3:.2f} Myr and lasts {off3-on3:.2f} Myr')
print(f"Core solidification begins at {results.at[0,'tsolid_start']:.2f} Ma and ends at {results.at[0,'tsolid']:.2f} Ma")

# Example run

## Temperature profile <a class="anchor" id="temp"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Create r array for plotting and prelog time to speed up plotting

In [ ]:
log_time = False

In [ ]:
rplot = np.arange(0,int(r)+dr,int(dr))/1e3
r_unstable=np.array([]) 
for ind in min_unstable:
    r_unstable = np.append(r_unstable,rplot[int(ind)])
rc = r/2

#log time
if log_time == True:
    tpt = np.log10(t_plot_t)
    tpa = np.log10(t_plot_all)
    lfcond = np.log10(fcond_t)
    on1l = np.log10(on1)
    on2l = np.log10(on2)
    on3l = np.log10(on3)
    off1l = np.log10(off1)
    off2l = np.log10(off3)
    off3l = np.log10(off3)
else: 
    tpt = t_plot_t
    tpa = t_plot_all
    lfcond = fcond_t
    on1l = on1
    on2l = on2
    on3l = on3
    off1l = off1
    off2l = off2
    off3l = off3

Make figure

In [ ]:
plt.figure(figsize=[10,20/3])
plt.pcolormesh(tpa[::2],rplot[::2],Tall[::2,::2],shading = 'gouraud',vmin=200,vmax=1600)
plt.hlines(rc/1e3,tpt[0],max(tpa),linestyle='--',color='black',label='CMB')
plt.vlines(tpt[0],0,r/1e3,linestyle='-.',label='Differentiation')
plt.fill_betweenx([0,rc/5e3],on1l,off1l,alpha=0,hatch='/',label='dynamo on')
plt.plot(tpt,r_unstable,linestyle='dotted',label='Convecting core')
if on2 > 0:
    plt.fill_betweenx([0,rc/5e3],on2l,off2l,alpha=0,hatch='/')
if on3 > 0: 
    plt.fill_betweenx([0,rc/5e3],on3l,off3l,alpha=0,hatch='/')

if np.any(t_plot_t<fcond_t):
    plt.plot(tpt[(t_plot_t<=fcond_t)&(d0<(r-rc))],(r-d0[(t_plot_t<=fcond_t)&(d0<(r-rc))])/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
    plt.plot(tpt[(t_plot_t<=fcond_t)&(r_unstable==0)][1:],(rc+dl[(t_plot_t<=fcond_t)&(r_unstable==0)][1:])/1e3,linestyle='dotted',label='top of $\delta_l$',color='blue')
    plt.vlines(tpt[t_plot_t<=fcond_t][-1],r/1e3,rc/1e3,linestyle='dotted',label='conductive mantle',color='red')
plt.plot(tpt[f<f0],f[f<f0]*rc/1e3,linestyle='-.',color='black',label='Top of liquid core')
#labels and limits
plt.ylabel('Distance from centre of planetesimal/km')
plt.xlabel('Time / Ma')
plt.colorbar(label='Temperature/K')
plt.ylim(bottom=0)
plt.legend(bbox_to_anchor=[1.6,0.5])
plt.xscale('log')
if save == True:
    plt.savefig(f'{save_path}run_{run}_thermal_profile.png',bbox_inches='tight',dpi=500)

How long is the planetesimal below 700K?

In [ ]:
tacc = 0.8 #accretion time in Ma
rlen = int((r/1e3)*2)
tsint = np.zeros([rlen])
for i in range(rlen):
    tsint[i]=tall[Tall[i,:]>700][0]
tshort = np.min(tsint/Myr-tacc)
print(f'The shortest time before sintering is {tshort:.2f} Ma \
and the longest time before sintering is {np.max(tsint/Myr-tacc):.2f} Ma')

In [ ]:
rcheck = rplot[:-1] #remove surface point
rsint = rcheck[(tsint/Myr)>(tshort+tacc)]
print(f'The inner {rsint[0]}km sinters within {tshort:.1f} Ma of accretion')

How long is the planetesimal above rcmf?

In [ ]:
tacc = 0.8 #accretion time in Ma
rlen = int((r/1e3))
trcmf = np.zeros([rlen])
rcmf = params['rcmf'].values[0]
for i in range(rlen):
    trcmf[i]=tall[Tall[nmantle+i,:]>(rcmf*(400)+1400)][-1]
max_rcmf = np.max(trcmf/Myr-tacc)
print(f'The shortest time before T<T_C is {np.min(trcmf/Myr-tacc):.2f} Ma \
and the longest time before T<T_C is {max_rcmf:.2f} Ma')

## Temperature and heat fluxes <a class="anchor" id="flux"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Have plotted only the central core temperature and convective mantle temperature.

In [ ]:
log_time = True

In [ ]:
import seaborn as sns

In [ ]:
dl_end = int(dl[Tm_conv==0][0]/dr) #index of CMB b.l. base above CMB at cessation of convection

with sns.plotting_context('paper',font_scale=1.4):
    plt.figure(tight_layout=True,figsize=[10,7])
    xmin=tstart

    #temperatures as function of time
    plt.subplot(2,1,1)

    plt.plot(t_plot_t,Tc,label='central core temperature',color='black')
    plt.plot(t_plot_t[Tm_conv!=0],Tm_conv[Tm_conv!=0],label='convective mantle temperature',color='#FF5350')
    plt.plot(t_plot_t[Tm_conv==0],T_profile[Tm_conv==0,nmantle+dl_end+2],label='conductive mantle temperature',color='#FF5350',linestyle='dashed')
    if log_time == True:
        plt.xscale('log')
    plt.ylim([1250,1550])
    plt.ylabel('T/K')
    plt.legend(loc='lower left')

    #fluxes as function of time
    plt.subplot(2,1,2)
    Fcmb_neg = Fcmb[Fcmb<0]
    Fcmb_pos = Fcmb[Fcmb>0]

    plt.semilogy(t_plot_t,Fs,label='$F_s$',color='#0F4C5C')
    plt.scatter(t_plot_t[Fcmb<0],abs(Fcmb_neg),label='$-F_{CMB}$',color='#5BC0EB',s=2)
    plt.scatter(t_plot_t[Fcmb>0],Fcmb_pos,label='$F_{CMB}$',color='#5F0F40',s=2)
    plt.semilogy(t_plot_t,Fad,label='$F_{ad}$',color='#E36414',linestyle='dashed')
    plt.semilogy(t_plot_t,Frad,label='$F_{rad}$',color='#9A031E',linestyle='-.')
    plt.fill_betweenx(y=[1e-4,100],x1=tstrat_remove,x2=terode,label='erosion of stratification',alpha=0.2)
    plt.fill_betweenx(y=[1e-4,100],x1=fcond_t,x2=max(t_plot_t),label='mantle conducting',alpha=0.2)
    plt.fill_betweenx(y=[1e-4,1e-3],x1=on1,x2=off1,label='dynamo on',alpha=0.2,color='grey')
    plt.fill_betweenx(y=[1e-4,1e-3],x1=on2,x2=off2,alpha=0.2,color='grey')
    if log_time == True:
        plt.xscale('log')
    plt.xlabel('Time/ Ma')

    plt.ylim([1e-4,1e2])   #use these limits when comparing runs
    plt.ylabel('Heat flux/ W$m^{-2}$')
    plt.legend(loc='upper right',ncol=2)

    if save == True:
        plt.savefig(f'{save_path}run_{run}_flux.png',dpi=500)

## Magnetic Field Strength <a class="anchor" id="Bfield"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Extract compositional and thermal buoyancy components

In [ ]:
comp = buoyr[0,:]
therm = buoyr[1,:]

In [ ]:
Xs_eutectic = 33
print(f'The core reaches the eutectic at {t_plot_t[Xs>=Xs_eutectic][0]:.2f} Ma')

Create data frames for rolling average compositional dynamo strengths

In [ ]:
compdf = pd.Series(comp[f<f0])
thermdf = pd.Series(therm[f<f0])
Remdf = pd.Series(Rem[f<f0])
Bdf = pd.Series(B[f<f0])

Make figure

There is a lag between the onset of solidification and the rolling average so plot the original time series and the average.

In [ ]:
wn = 75 #averaging window width
fig, ax = plt.subplots(nrows=2,ncols=1,sharex='col',figsize=[10,8])
#B and Rem
ax2 = ax[0].twinx()
ln1 = ax[0].plot(t_plot_t[f>=f0],np.ma.masked_where(Rem[f>=f0]<10,B[f>=f0]),color='black')
ax[0].plot(t_plot_t[f<f0],np.ma.masked_where(Rem[f<f0]<10,B[f<f0]),color='black',alpha=0.1) #original series
ax[0].plot(t_plot_t[f<f0],Bdf.rolling(window=wn,center=True).mean(),color='black') #average
ln2 = ax2.plot(t_plot_t[f>=f0],Rem[f>=f0],color='darkorchid')
ax2.plot(t_plot_t[f<f0],Rem[f<f0],color='darkorchid',alpha=0.1) #original series
ax2.plot(t_plot_t[f<f0],Remdf.rolling(window=wn,center=True).mean(),color='darkorchid') #average

ax2.hlines(10,min(t_plot_t),max(t_plot_t),linestyle='dashed',color='grey')
ax[0].legend(ln1+ln2,['Magnetic field strength','Re$_m$'])
ax[0].set_ylabel('Magnetic field strength/ $\mu T$')
ax2.set_ylabel('$Re_m$')
ax[0].tick_params(axis='y',colors='black')
ax2.tick_params(axis='y',colors='darkorchid')
ax[0].yaxis.label.set_color('black') 
ax2.yaxis.label.set_color('darkorchid') 

#thermal and compositional buoyancy flux
ln3 = ax[1].plot(t_plot_t[f>=f0],therm[f>=f0],color='black')
#ax[1].plot(t_plot_t[f<f0],therm[f<f0],color='black',alpha=0.1)
ax[1].plot(t_plot_t[f<f0],thermdf.rolling(window=wn,center=True).mean(),color='black')
ax[1].plot(t_plot_t[f<f0],comp[f<f0],color='darkorchid',alpha=0.1)
ln4=ax[1].plot(t_plot_t[f<f0],compdf.rolling(window=wn,center=True).mean(),color='darkorchid')
ax[1].legend(ln3+ln4,['thermal','compositional'])
ax[1].set(xscale='log',ylim=[1e-11,1e-8],ylabel='Buoyancy flux per unit area /kg$s^{-1}m^{-2}$',xlabel='Time/Ma')

if save == True:
    plt.savefig(f'{save_path}run_{run}_Bbuoy.pdf',dpi=500)

For the 500km body the buoyancy flux ratio is not useful, because of the cessation of mantle convection in the middle, but can add the code below to the plot abovr if you want to plot it.

### What is the peak field strength in each epoch?
Put in times by eye from plot above

In [ ]:
tsolid_start = results.at[0,'tsolid_start']
peakB = []
peakB.append(np.max(B[t_plot_t<10]))
peakB.append(np.max(B[(t_plot_t>10)&(t_plot_t<200)]))
peakB.append(np.max(B[(t_plot_t>200)&(t_plot_t<tsolid_start)]))
peakB.append(np.max(Bdf.rolling(window=wn,center=True).mean()))
for i, Bval in enumerate(peakB):
    print(f"The peak field strength in regime {i+1} is {Bval:.2f} muT")

### What causes the different trends in field generation?

Early times

Find the index of the time series that corresponds to the xlim on the time axis to speed up plotting

In [ ]:
xlim_up = 40 #40 Ma
tind = np.where(t_plot_t<xlim_up)[-1][-1]

In [ ]:
ticks = np.arange(1500,1530,10)
labels = (ticks-1400)/400 #melt fraction
fig, ax = plt.subplots(1,1)
ax1 = ax.twinx()
ln2 = ax.plot(t_plot_t,dl/1e3,color='darkorchid',linestyle='dashed')
mappable = ax.pcolormesh(t_plot_t[:tind],rplot[:11],np.transpose(T_profile[:(tind-1),(nmantle+1):(nmantle+11)]),vmin=1500,vmax=1520)
ln1 = ax1.plot(t_plot_t,Tcmb-Tm_conv,color='black')
ax1.set(ylim=[0,10],xscale='log',xlim=[diff_time,xlim_up],xlabel='Time/Ma',ylabel='$T_{CMB}-T_{m,conv}$/K')
ax.set(ylabel='$\delta_l$/km',ylim=[0,5])
ax.tick_params(axis='y',colors='darkorchid') 
ax.yaxis.label.set_color('darkorchid') 
ax1.legend(ln1+ln2,['$T_{CMB}-T_{m,conv}$','$\delta_l$'])
cax = fig.add_axes([1,0.15,0.025,0.7])
col = fig.colorbar(mappable,cax=cax,orientation='vertical',label='Melt fraction')
col.set_ticks(ticks, labels=labels)
if save == True:
    plt.savefig(f'{save_path}run_{run}_deltal.png',dpi=500)

Epoch 1 (until 14Ma): lid thickens so $F_{CMB}$ decreases. There is a jump in lid thickness and decrease in $F_{CMB}$ at 14Ma after CAI when the base of the mantle goes below $\phi_C$. This leads to an increased temperature difference between the core and mantle, which counteracts the increase in viscosity is balanced to give the decrease in $\delta_l$ from 14-20Ma before lid thickness then increases again.

Late times - why does strength increase before the onset of core solidification?

In [ ]:
fig, ax = plt.subplots(1,1)
ln1 = ax.plot(t_plot_t,Tcmb-T_profile[:,nmantle+1],color='black')
ax.set(ylim=[0,3],xlim=[on2,tsolid_start],xlabel='Time/Ma',ylabel='$T_{CMB}-T_{m,CMB+1}$ /K')

Temperature difference between the CMB and one cell above the CMB increases, which drives the increase in $F_{CMB}$ and field strength. This is due to increased efficiency of mantle cooling as shallower layers of the mantle cool more.

## Endmember strength comparison <a class="anchor" id="icfrac"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Import other endmember field strengths

In [ ]:
run = 3
if automated == True:
    r_run2, dr2, tstart2, dt2, viscosity2, icfrac2 = load_run_info(run,f'{path}auto_params.csv')
else:
    r_run2, dr2, tstart2, dt2, viscosity2, icfrac2 = load_run_info(run,f'{path}run_info.csv')
npzfile = np.load(f'{path}run_{run}.npz')

f2 = npzfile['f'] 
t2 = npzfile['t'] #time in s
Rem2 = npzfile['Rem'] # magnetic Reynolds number 
B2 = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr2 = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Xs2 = npzfile['Xs']
t_plot_t2 = t2/Myr

In [ ]:
comp2 = buoyr2[0,:]
therm2 = buoyr2[1,:]

In [ ]:
compdf2 = pd.Series(comp2[f2<f0])
thermdf2 = pd.Series(therm2[f2<f0])
Remdf2 = pd.Series(Rem2[f2<f0])
Bdf2 = pd.Series(B2[f2<f0])

Only plot compositional dynamo

In [ ]:
wn = 200 #averaging window width
fig, ax = plt.subplots(nrows=1,ncols=2,sharex='col',figsize=[10,3])
#B and Rem
t_plot_solid = t_plot_t[f<f0]
t_plot_solid2 = t_plot_t2[f2<f0]
# B - icfrac = 0
#ln1 = ax[0].plot(t_plot_t[f>=f0],np.ma.masked_where(Rem[f>=f0]<10,B[f>=f0]),color='#0000ca')
ln1 = ax[0].plot(np.ma.masked_where(Remdf.rolling(window=wn,center=True).mean()<10,t_plot_solid),\
                 np.ma.masked_where(Remdf.rolling(window=wn,center=True).mean()<10,Bdf.rolling(window=wn,center=True).mean()),\
                 color='#0000ca')
#icfrac = 1
#ln2 = ax[0].plot(t_plot_t2[f2>=f0],np.ma.masked_where(Rem2[f2>=f0]<10,B2[f2>=f0]),color='#0079cb')
ln2 = ax[0].plot(np.ma.masked_where(Remdf2.rolling(window=wn,center=True).mean()<10,t_plot_solid2),\
                 np.ma.masked_where(Remdf2.rolling(window=wn,center=True).mean()<10,Bdf2.rolling(window=wn,center=True).mean()),\
                 color='#0079cb')
# Rem  icfrac = 0
#ln3 = ax[1].plot(t_plot_t[f>=f0],Rem[f>=f0],color='#0000ca')
ln3 = ax[1].plot(t_plot_t[f<f0],Remdf.rolling(window=wn,center=True).mean(),color='#0000ca')
#icfrac = 1
#ln4 = ax[1].plot(t_plot_t2[f2>=f0],Rem2[f2>=f0],color='#0079cb')
ln4 = ax[1].plot(t_plot_t2[f2<f0],Remdf2.rolling(window=wn,center=True).mean(),color='#0079cb')
ax[1].hlines(10,min(t_plot_t[f<f0]),max(t_plot_t),linestyle='dashed',color='grey')
ax[0].legend(ln1+ln2,[f'$m_{{frac}}$={icfrac:.0g}',f'$m_{{frac}}$={icfrac2:.0g}'])
#ax[1].legend(ln3+ln4,[f'$m_{{frac}}$={icfrac:.0g}',f'$m_{{frac}}$={icfrac2:.0g}'])
ax[0].set(ylabel='Magnetic field \n strength/ $\mu T$',xlabel='Time /Ma',xlim=[350,575])
ax[1].set(xlabel='Time /Ma',ylabel='$Re_m$',xlim=[350,575])
if save == True:
    plt.savefig(f'{save_path}Bendmember.png',dpi=500,bbox_inches='tight')

There is negligible difference in field strength and $Re_m$ between the two endmembers. Increasing $m_{frac}$, increases field strength but decreases $Re_m$.

# Discussion

## Convective lengthscales <a class="anchor" id="conv"></a>
<p align="right">(<a href="#top">back to top</a>)</p>
Justify convective lengthscales assumptions. 

Define functions for r1 and r2

In [ ]:
@np.vectorize
def r1(x,f):
    """
    outer solid shell inner radius

    Parameters
    ----------
    x : float
        fraction of mass in outer shell
    f : float
        fractional inner core radius for concentric inward

    Returns
    -------
    r1 : float
        inner radius of outer solid shell

    """
    r1 = rc*(1-(1-x)*(1-f**3))**(1/3)
    return r1

@np.vectorize
def r2(x,f):
    """
    inner solid core radius

    Parameters
    ----------
    x : float
        fraction of mass in outer shell
    f : float
        fractional inner core radius for concentric inward

    Returns
    -------
    r2 : float
        inner solid core radius

    """
    r2 = rc*(x*(1-f**3))**(1/3)
    return r2

In [ ]:
r1val1 = r1(icfrac,f[(Xs<Xs_eutectic)&(f<f0)])/rc
r2val1 = r2(icfrac,f[(Xs<Xs_eutectic)&(f<f0)])/rc
r1val2 = r1(icfrac2,f2[(Xs2<Xs_eutectic)&(f2<f0)])/rc
r2val2 = r2(icfrac2,f2[(Xs2<Xs_eutectic)&(f2<f0)])/rc
#estimate for x = 0.5
r1val3 = r1(0.5,f2[(Xs<Xs_eutectic)&(f<f0)])/rc
r2val3 = r2(0.5,f2[(Xs<Xs_eutectic)&(f<f0)])/rc

Make plots - make sure first and second run simported are 27.1 wt%

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax1 = ax.twinx()
c2 = '#0079cb'
# x = 0
ax.plot(t_plot_t[(Xs<Xs_eutectic)&(f<f0)],r1val2-r2val2,label=f'$m_{{frac}}$={icfrac2}',color='#0000ca')
ax1.plot(t_plot_t[(Xs<Xs_eutectic)&(f<f0)],r1val2/f2[(Xs2<Xs_eutectic)&(f2<f0)],color=c2)
# x = 1
ax.plot(t_plot_t[(Xs<Xs_eutectic)&(f<f0)],r1val1-r2val1,label=f'$m_{{frac}}$={icfrac}',color='#0000ca',linestyle='dashed')
ax1.plot(t_plot_t[(Xs<Xs_eutectic)&(f<f0)],r1val1/f[(Xs<Xs_eutectic)&(f<f0)],linestyle='dashed',color=c2)

ax.set_xlabel('Time /Ma')
ax.set_ylabel('Convective lengthscale /r$_c$')
ax1.set_ylabel('$r_1$/$r_i$ ')
ax1.tick_params(axis='y',colors=c2) 
ax1.yaxis.label.set_color(c2) 
ax.legend(loc='center left')
ax.set_ylim([0,1])
if save == True:
    plt.savefig(f'{save_path}convl.png',dpi=450)

There is a 5% difference between $r_i$ (inner core radius for concentric inward) and $r_1$ (outer core radius with passive inner core) even for the endmember where all the mass ends up in the passive inner core so it is reasonable to approximate them as the same.

The overall convective lengthscale is reduced by a factor of 0.6 from x=0 to x=1.


### Ratio of endmember field strengths and $Re_m$ <a class="anchor" id="ratio"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Test the predicted ratio of convective lengthscale l to ratio of B or Rem, given by the gradient of a log-log plot.

Expect $Re_m \propto l^{0.32}$ and $B \propto l$

This plot is for the supplementary.

In [ ]:
wn = 200 #averaging window width
fig, ax = plt.subplots(nrows=2,ncols=1,sharex='col',figsize=[10,6])
#B and Rem

max_ind = len(f[(Xs<Xs_eutectic)&(f<f0)])
#data
ln1 = ax[0].plot(((r1val1-r2val1)/(r1val2-r2val2)),(Bdf.rolling(window=wn,center=True).mean()/Bdf2.rolling(window=wn,center=True).mean())[:max_ind],color='#0000ca')
ln2 = ax[1].plot(((r1val1-r2val1)/(r1val2-r2val2)),(Remdf.rolling(window=wn,center=True).mean()/Remdf2.rolling(window=wn,center=True).mean())[:max_ind],color='#0000ca')
#proposed relationship - only plot after the Nan added by the rolling mean function
ln3 = ax[0].plot(((r1val1-r2val1)/(r1val2-r2val2))[int(wn/2):],(((r1val1-r2val1)/(r1val2-r2val2))**(-0.24))[int(wn/2):],color='#0079cb',linestyle='dashed')
ln4 = ax[1].plot(((r1val1-r2val1)/(r1val2-r2val2))[int(wn/2):],(((r1val1-r2val1)/(r1val2-r2val2))**(0.32))[int(wn/2):],linestyle='dashed',color='#0079cb')

ax[0].set_ylabel('$B_0/B_1$')
ax[1].set_ylabel('$Re_{m,0}/Re_{m,1}$')
ax[1].set_xlabel('$l_0/l_1$')

#ax[0].set_xscale('log')
#ax[0].set_yscale('log')
#ax[1].set_yscale('log')

ax[0].legend(ln1+ln3,['Model output','$(B_0}$/$B_1$ $\propto$ $l_0$/$l_1)^{-0.24}$'])
ax[1].legend(ln2+ln4,['Model output','$Re_{m,0}/Re_{m,1} \propto (l_0/l_1)^{0.32}$'])
if save == True:
    plt.savefig(f'{save_path}Bratios.png',dpi=450)

## Thermal vs compositional dynamos  <a class="anchor" id="therm-comp"></a>
<p align="right">(<a href="#top">back to top</a>)</p>
Compare three runs with same geometry, but different sulfur contents. Run 6 solidification is eutectic so doesn't generate buoyancy.

Import 27 wt% run

In [ ]:
run = 2
npzfile = np.load(f'{path}run_{run}.npz')

f3 = npzfile['f'] 
t3 = npzfile['t'] #time in s
Rem3 = npzfile['Rem'] # magnetic Reynolds number 
B3 = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr3 = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Xs3 = npzfile['Xs']
t_plot_t3 = t3/Myr

In [ ]:
comp3 = buoyr3[0,:]
therm3 = buoyr3[1,:]

In [ ]:
compdf3 = pd.Series(comp3[f3<f0])
thermdf3 = pd.Series(therm3[f3<f0])
Remdf3 = pd.Series(Rem3[f3<f0])
Bdf3 = pd.Series(B3[f3<f0])

Import eutectic run

In [ ]:
run = 6

npzfile = np.load(f'{path}run_{run}.npz')

f4 = npzfile['f'] 
t4 = npzfile['t'] #time in s
Rem4 = npzfile['Rem'] # magnetic Reynolds number 
B4 = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr4 = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Xs4 = npzfile['Xs']
t_plot_t4 = t4/Myr

In [ ]:
wn = 200 #averaging window width
c2= '#DAC800'
c1 = '#CA7800' #highest S content
c3='#4F4F4F' #lowest S content
fig, ax = plt.subplots(nrows=2,ncols=1,figsize=[10,6],sharex='col')
#scale solidification time
t_plot_solid = t_plot_t[f<f0]
t_plot_solid3 = t_plot_t3[f3<f0]

# B 
#non-eutectic - 27.1wt%
ln0 = ax[0].plot(t_plot_t3[f3>=f0],np.ma.masked_where(Rem3[f3>=f0]<10,B3[f3>=f0]),color=c3)
ax[0].plot(t_plot_t3[f3<f0],np.ma.masked_where(Rem3[f3<f0]<10,B3[f3<f0]),color=c3,alpha=0.1) #original series
ax[0].plot(np.ma.masked_where(Remdf3.rolling(window=wn,center=True).mean()<10,t_plot_solid3),\
                 np.ma.masked_where(Remdf3.rolling(window=wn,center=True).mean()<10,Bdf3.rolling(window=wn,center=True).mean()),\
                 color=c3)
#non-eutectic - 30.05wt%
ax[0].plot(t_plot_t[f>=f0],np.ma.masked_where(Rem[f>=f0]<10,B[f>=f0]),color=c1)
ax[0].plot(t_plot_t[f<f0],np.ma.masked_where(Rem[f<f0]<10,B[f<f0]),color=c1,alpha=0.1) #original series
ln1 = ax[0].plot(np.ma.masked_where(Remdf.rolling(window=wn,center=True).mean()<10,t_plot_solid),\
                 np.ma.masked_where(Remdf.rolling(window=wn,center=True).mean()<10,Bdf.rolling(window=wn,center=True).mean()),\
                 color=c1)
#eutectic
ln2 = ax[0].plot(t_plot_t4,np.ma.masked_where(Rem4<10,B4),color=c2) 

#Rem
#non-eutectic - 27.1 wt%
ax[1].plot(t_plot_t3[f3>=f0],Rem3[f3>=f0],color=c3)
ax[1].plot(t_plot_solid3, Remdf3.rolling(window=wn,center=True).mean(),color=c3)
ax[1].plot(t_plot_t3[f3<f0],Rem3[f3<f0],color=c3,alpha=0.1) #original series
#non-eutectic - 30.05 wt%
ax[1].plot(t_plot_t[f>=f0],Rem[f>=f0],color=c1)
ax[1].plot(t_plot_solid, Remdf.rolling(window=wn,center=True).mean(),color=c1)
ax[1].plot(t_plot_t[f<f0],Rem[f<f0],color=c1,alpha=0.1) #original series
#eutectic
ax[1].plot(t_plot_t4,Rem4,color=c2) 

ax[1].hlines(10,min(t_plot_t),max(t_plot_t),linestyle='dashed',color='grey')
ax[0].legend(ln0+ln1+ln2,[f'X$_{{s,0}}$={Xs3[0]:.1f} wt%',f'X$_{{s,0}}$={Xs[0]:.1f} wt%','Eutectic $X_{S,0}$'],loc='lower left')
ax[0].set(ylabel='Magnetic field \n strength/ $\mu T$',ylim=[0,20])
ax[0].set(xlabel='Time /Ma')
ax[1].set(xlabel='Time /Ma',ylabel='$Re_m$',xscale='log',ylim=[0,150])
if save == True:
    plt.savefig(f'{save_path}Bnobuoy.pdf',dpi=500)

What is the difference in field strength when the compositional field is maximal?

In [ ]:
np.max(Bdf3.rolling(window=wn,center=True).mean())/B4[t_plot_t4>t_plot_solid3[0]][0]